## 이미지 전처리 -> 학습 ?

# 학습용 데이터 딥러닝

In [1]:
import os, cv2, dlib
import matplotlib.pyplot as plt
import numpy as np
from pandas import DataFrame, Series
import pandas as pd
from PIL import Image

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

image_path_dir = 'c:/MyJupyter/Image_analysis/images/'
emotion = ['angry', 'disgust', 'fear', 'happy', 
          'neutral', 'sad', 'surprise']
emotion_dir = []
emotion_list = []
one_emotion = []
image_number = 0

for e in emotion:
    emotion_dir.append(image_path_dir + e + '/')
    
for e_dir in emotion_dir:
    # 한 감정에 따른 사진 전부 읽기
    one_emotion = os.listdir(e_dir)
    image_list = []
    # 이미지 전부 담기
    for one_image in one_emotion:
        # 이미지 하나 읽어오기
        image = cv2.imread(e_dir + one_image)
        
        # 못 읽는 이미지 넘어가기
        if np.shape(image) == (): pass
        else :
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

            # 얼굴 탐색
            faces = detector(image)

            # 얼굴 탐지 실패시 for문 안들어감
            for face in faces:
                # 얼굴의 좌표값이 나온다
                x1 = face.left()
                y1 = face.top()
                x2 = face.right()
                y2 = face.bottom()

                # 얼굴만 잘라서 사진 저장
                face_image = image.copy();
                face_image = face_image[y1:y2 , x1:x2]
                save_dir = 'c:/MyJupyter/Image_analysis/mine/'
                save = save_dir + str(image_number) + ".jpg"
                image_number += 1
                cv2.imwrite(save, face_image)

                image_list.append(face_image)
    emotion_list.append(image_list)


emotion_list = np.array(emotion_list)
emotion_index = 0
dic = []
df = pd.DataFrame()
for e_list in emotion_list:
    data = {'face' : e_list, "emotion" : emotion[emotion_index]}
    dic = pd.DataFrame.from_dict(data)
    df = pd.concat([df,dic])
    emotion_index += 1

# df.columns = emotion, emotion result 2가지
# emotion_list에 7가지 감정을 분류하여 다 들어갔음.
# angry, disgust, fear, happy, neutral, sad, surprise 순서


df

,face,emotion
0,"[[11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, ...",angry
1,"[[68, 121, 58, 64, 71, 63, 137, 98, 26, 61, 96...",angry
2,"[[121, 121, 124, 125, 109, 71, 39, 56, 65, 73,...",angry
3,"[[191, 212, 212, 210, 205, 204, 199, 193, 193,...",angry
4,"[[48, 41, 38, 41, 47, 49, 51, 53, 59, 65, 66, ...",angry
...,...,...
13,"[[152, 154, 155, 155, 153, 151, 149, 153, 149,...",surprise
14,"[[59, 67, 83, 90, 99, 110, 116, 120, 125, 134,...",surprise
15,"[[93, 91, 108, 78, 63, 51, 60, 73, 71, 101, 13...",surprise
16,[],surprise


In [35]:
df.head(2)

,emotion,result
0,"[[11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, ...",angry
1,"[[68, 121, 58, 64, 71, 63, 137, 98, 26, 61, 96...",angry


리스트를 읽어와 이미지 경로 대신에
전처리 된 값을 'emotion' 에 넣어주면 된다

## 이미지 벡터로 바꾸기

In [40]:
import keras
from keras import layers
from keras import models



# 이미지 늘리기
## ImageDataGenerator 클래스

In [4]:
import numpy as np
import keras
from keras import layers, models, datasets, backend
from keras.utils import np_utils
import matplotlib.pyplot as plt
from sklearn import model_selection
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    # 0~180의 값, 임의로 그림을 회전
        rotation_range=40,
    # width or height shift = 임의로 수직, 수평이미지를 변환
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
    # 전단 변형을 부작위로 적용하기 위한 것
        shear_range=0.2,
    # 사진 내부를 무작위로 확대하기 위한 것
        zoom_range=0.2,
    # 이미지의 절반을 가로로 무작위로 뒤집기 위한 것
        horizontal_flip=True,
    # 회전 또는 너비/높이 이동 후 나타날 수 있는 새로 만든 픽셀을 채우는 데 사용
        fill_mode='nearest'
        )